In [1]:
import numpy as np
import os
import tensorflow as tf 
# import keras.backend as K

In [2]:
x_train = np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/x_upsampled.npy") 
y_train = np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/Features/NCAR/_20.npy")
x_test = np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/x_test.npy")
y_test = np.load("C:\\Users\\gazur\\Desktop\\Computer Vision\\Imbalance medical image synthesis with label noise\\DATASET\\Finale_Data/y_test.npy")

In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1627, 256, 256, 3), (1627, 2), (379, 256, 256, 3), (379, 2))

In [4]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5)

Using TensorFlow backend.


In [5]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=(256,256,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(256,256,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(2))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 96)        384       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 256)       1024      
______________________________________

In [6]:
import time

In [7]:
AlexNet.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adagrad(learning_rate=0.001),metrics=['accuracy'])

# start_time = time.time()
# AlexNet.fit(x_train, y_train, batch_size=16, epochs=1, callbacks = [lrr], verbose=1)
# score = AlexNet.evaluate(x_test, y_test, batch_size=32)
# end_time = time.time()
# print("--- Time taken to train : %s minuts ---" % ((end_time - start_time)//60))

In [8]:
start_time = time.time()
AlexNet.fit(x_train, y_train,
            batch_size=16,
            epochs=10,
            validation_data=(x_test, y_test),
            callbacks=[lrr], # early_stopping, checkpointer, reduce_lr
            shuffle=False)
end_time = time.time()
print("--- Time taken to train : %s minuts ---" % ((end_time - start_time)//60))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1627 samples, validate on 379 samples
Epoch 1/10
1627/1627 [==============================] - 153s 94ms/step - loss: 0.7118 - accuracy: 0.5900 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 2/10


C:\Anaconda\envs\GPU_Keras\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1627/1627 [==============================] - 139s 86ms/step - loss: 0.6306 - accuracy: 0.6724 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 3/10
1627/1627 [==============================] - 142s 87ms/step - loss: 0.6018 - accuracy: 0.6921 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 4/10
1627/1627 [==============================] - 142s 87ms/step - loss: 0.5579 - accuracy: 0.7265 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 5/10
1627/1627 [==============================] - 141s 87ms/step - loss: 0.5142 - accuracy: 0.7554 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 6/10
1627/1627 [==============================] - 144s 89ms/step - loss: 0.4498 - accuracy: 0.8021 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 7/10
1627/1627 [==============================] - 151s 93ms/step - loss: 0.3807 - accuracy: 0.8519 - val_loss: 3.0350 - val_accuracy: 0.8021
Epoch 8/10
1627/1627 [==============================] - 149s 92ms/step - loss: 0.3166 - accuracy: 0.8888 - val_loss: 3.0350 - val